In [ ]:
from glob import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
def voxel_volume(r1, r2, theta1, theta2):
    volume = 2/3 * np.pi * (r2**3 - r1**3) * (np.cos(theta1) - np.cos(theta2))
    return volume

In [ ]:
radii_filepath = '../../../data/collapsedcone/kernels/original/voxels.dat'
loaded_radii_boundaries = pd.read_csv(radii_filepath, header=None)[0].values

radii_boundaries = np.concatenate([[0], loaded_radii_boundaries])
radii = radii_boundaries[0:-1] + (radii_boundaries[1::] - radii_boundaries[0:-1]) / 2

In [ ]:
dtheta = np.pi / 48
angle_boundaries = np.arange(0, np.pi + dtheta, dtheta)

In [ ]:
degrees = np.arange(180/48/2, 180, 180/48)

In [ ]:
theta_mesh, radii_mesh = np.meshgrid(degrees/180 *np.pi, radii)

xx = radii_mesh * np.cos(theta_mesh)
yy = radii_mesh * np.sin(theta_mesh)

In [ ]:
voxel_volumes_mesh = np.array([
    [
        voxel_volume(
            radii_boundaries[i], radii_boundaries[i+1], 
            angle_boundaries[j], angle_boundaries[j+1])
        for j in range(len(angle_boundaries) - 1)
    ] for i in range(len(radii_boundaries) - 1)
])

np.shape(voxel_volumes_mesh)

In [ ]:
voxel_mass_mesh = voxel_volumes_mesh / 1000  # cm^3 of water --> kg

In [ ]:
len(radii_boundaries)

In [ ]:
np.shape(theta_mesh)

In [ ]:
glob('../../../data/collapsedcone/kernels/original/scaf*')

In [ ]:
kernel_filepath = '../../../data/collapsedcone/kernels/original/scaf1250'
kernel_file = pd.read_csv(kernel_filepath, header=None, delimiter='\s+')

In [ ]:
mev_to_joule = 1.6021773e-13
joules = 1.25 * mev_to_joule
joules

In [ ]:
index_map = {
    0: 'primary',
    1: 'first_scatter',
    2: 'second_scatter',
    3: 'multiple_scatter',
    4: 'bremsstrahlung_and_annihilation',
    5: 'mean_radius',
    6: 'mean_angle'
}

def get_item_from_kernel_file(index):
    column_1 = kernel_file.iloc[index::7, 0].values
    
    return np.reshape(column_1, (len(degrees), len(radii))).T

# kernel_data = {
#     index_map[index]: get_item_from_kernel_file(index)
#     for index in index_map
# }

# total_dose = (
#     kernel_data['primary'] +
#     kernel_data['first_scatter'] + 
#     kernel_data['second_scatter'] +
#     kernel_data['multiple_scatter'] +
#     kernel_data['bremsstrahlung_and_annihilation']
# ) * joules / voxel_mass_mesh

In [ ]:
dose_index = range(5)

dose_data = {
    index_map[index]: get_item_from_kernel_file(index) * joules / voxel_mass_mesh
    for index in dose_index
}

total_dose = (
    dose_data['primary'] +
    dose_data['first_scatter'] + 
    dose_data['second_scatter'] +
    dose_data['multiple_scatter'] +
    dose_data['bremsstrahlung_and_annihilation']
)

voxel_index = range(5,7)

voxel_data = {
    index_map[index]: get_item_from_kernel_file(index)
    for index in voxel_index
}

In [ ]:
paper_data = {
    index_map[index]: get_item_from_kernel_file(index) * mev_to_joule / voxel_mass_mesh * 100  # cGy / MeV / photon
    for index in dose_index
}

total_dose_paper_data = (
    paper_data['primary'] +
    paper_data['first_scatter'] + 
    paper_data['second_scatter'] +
    paper_data['multiple_scatter'] +
    paper_data['bremsstrahlung_and_annihilation']
)

In [ ]:
plt.scatter(xx, yy, c=np.log10(paper_data['primary']), s=5)
plt.contour(xx, yy, np.log10(paper_data['primary']), levels=[-11, -10, -9, -8])
plt.colorbar()
plt.axis('equal');
plt.xlim([-0.5,0.5])
plt.ylim([0,0.5])
plt.title('Replicating Figure 4')

In [ ]:
plt.scatter(xx, yy, c=np.log10(paper_data['first_scatter']), s=5)
plt.contour(xx, yy, np.log10(paper_data['first_scatter']), levels=[-13, -12, -11, -10])
plt.colorbar()
plt.axis('equal');
plt.xlim([-10,10])
plt.ylim([0,10])
plt.title('Replicating Figure 5')

In [ ]:
plt.scatter(xx, yy, c=np.log10(total_dose_paper_data), s=5)
plt.contour(xx, yy, np.log10(total_dose_paper_data), levels=[-13, -12, -11, -10])
plt.colorbar()
plt.axis('equal');
plt.xlim([-10,10])
plt.ylim([0,10])
plt.title('Replicating Figure 6')

In [ ]:
to_plot = [
    'primary', 'first_scatter',
    'second_scatter', 'multiple_scatter',
    'bremsstrahlung_and_annihilation']

In [ ]:
for i, degree in enumerate(degrees):
    plt.figure()
    plt.title(degree)
    
    for key in to_plot:
        plt.plot(radii, dose_data[key][:, i], 'o-', label=key)
        plt.xlim([0, 1])
    
    plt.legend()
    
    plt.show()

In [ ]:
for i, radius in enumerate(radii):
    plt.figure()
    plt.title(radius)
    
    for key in to_plot:
#         plt.plot(voxel_data['mean_angle'][i, :], dose_data[key][i, :], 'o-', label=key)
        plt.plot(degrees, dose_data[key][i, :], 'o-', label=key)
    
    plt.legend()
    
    plt.show()